# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
pip install gmaps

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
%matplotlib inline

# Import API key
from api_keys import g_key

In [3]:
gmaps.configure(g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:
csv_path = pd.read_csv('city_data.csv')
df = pd.DataFrame(csv_path)
del df['Unnamed: 0']
df.head()

,city,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Hobart,-42.8794,147.3294,52.27,93,75,9.22,AU,1651014725
1,Pechenga,69.5259,31.1703,18.43,69,100,24.27,RU,1651014726
2,Rikitea,-23.1203,-134.9692,78.22,69,100,20.49,PF,1651014217
3,Hermanus,-34.4187,19.2345,56.79,77,0,9.22,ZA,1651014646
4,Lavrentiya,65.5833,-171.0000,25.86,83,50,3.67,RU,1651014727


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
humidity = df["Humidity"].astype(float)
maxhumidity = humidity.max()
locations = df[["Lat", "Lng"]]

In [7]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=maxhumidity,point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [11]:
narrowed_df = df.loc[(df["Max Temp"] > 70) & (df["Max Temp"] < 80) & (df["Cloudiness"] == 0) & (df["Wind Speed"] < 10), :]
narrowed_df = narrowed_df.dropna(how='any')
narrowed_df.reset_index(inplace=True)
del narrowed_df['index']
narrowed_df.head()

,city,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Tura,25.5198,90.2201,71.91,92,0,5.48,IN,1651014839
1,Caravelas,-17.7125,-39.2481,75.76,81,0,5.30,BR,1651014922
2,Atar,20.5169,-13.0499,74.50,31,0,9.86,MR,1651014897
3,Fazilpur,32.1833,75.0667,77.50,24,0,3.02,PK,1651015073
4,São Miguel do Araguaia,-13.2750,-50.1628,75.63,59,0,4.38,BR,1651015116


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [14]:
hotels = []

for i in range(len(narrowed_df)):
    lat = narrowed_df.loc[i]['Lat']
    lng = narrowed_df.loc[i]['Lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(url, params=params)
    requested.json = requested.json()
    try:
        hotels.append(requested.json['results'][0]['name'])
    except:
        hotels.append("")
narrowed_df["Hotel Name"] = hotels
narrowed_df = narrowed_df.dropna(how='any')
narrowed_df.head()

,city,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Tura,25.5198,90.2201,71.91,92,0,5.48,IN,1651014839,Tura
1,Caravelas,-17.7125,-39.2481,75.76,81,0,5.30,BR,1651014922,Caravelas
2,Atar,20.5169,-13.0499,74.50,31,0,9.86,MR,1651014897,Atar
3,Fazilpur,32.1833,75.0667,77.50,24,0,3.02,PK,1651015073,Chak Qazian
4,São Miguel do Araguaia,-13.2750,-50.1628,75.63,59,0,4.38,BR,1651015116,São Miguel do Araguaia


In [22]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>city</dt><dd>{city}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_df.iterrows()]
locations = narrowed_df[["Lat", "Lng"]]

In [23]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))